In [1]:
#Modules
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
# Load data
df = pd.read_csv("Market Prices Search.csv")

# View first rows
print(df.head())


             Market  Commodity Classification Grade Sex Wholesale     Retail  \
0           Gikomba  Dry Maize   Yellow Maize     -   -  77.78/Kg  100.00/Kg   
1           Gikomba  Dry Maize    White Maize     -   -  50.00/Kg   60.00/Kg   
2          Kerugoya  Dry Maize    White Maize     -   -  40.00/Kg   65.00/Kg   
3  Ngurubani Market  Dry Maize    White Maize     -   -  46.67/Kg   50.00/Kg   
4           Gikomba  Dry Maize   Yellow Maize     -   -  77.78/Kg  100.00/Kg   

   Supply Volume     County        Date  
0            NaN    Nairobi  2025-12-16  
1            NaN    Nairobi  2025-12-16  
2         4500.0  Kirinyaga  2025-12-15  
3         5450.0  Kirinyaga  2025-12-15  
4            NaN    Nairobi  2025-12-15  


,Supply Volume
count,1281.000000
mean,5553.990632
std,5518.131324
min,0.000000
25%,3000.000000
50%,5000.000000
75%,6300.000000
max,80000.000000
